In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

2025-05-24 12:53:05.980148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748109186.002790    9041 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748109186.009738    9041 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748109186.027273    9041 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748109186.027293    9041 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748109186.027295    9041 computation_placer.cc:177] computation placer alr

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0


In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, 3, padding="valid", activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation="relu"),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(10),
    ]
)

I0000 00:00:1748109192.572403    9041 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15494 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:02:00.0, compute capability: 6.0
I0000 00:00:1748109192.572955    9041 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 15506 MB memory:  -> device: 1, name: Tesla P100-PCIE-16GB, pci bus id: 0000:82:00.0, compute capability: 6.0


In [4]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,034 (879.04 KB)

 Trainable params: 225,034 (879.04 KB)

 Non-trainable params: 0 (0.00 B)

None


In [5]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [6]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10


I0000 00:00:1748109195.939829    9297 service.cc:152] XLA service 0x2aaec000a1a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748109195.940318    9297 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1748109195.940323    9297 service.cc:160]   StreamExecutor device (1): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-05-24 12:53:16.085785: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1748109196.426544    9297 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-05-24 12:53:17.718178: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,30,30]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.2231, f32[32,3,3,3]{3,2,1,0} %bit

782/782 - 8s - 11ms/step - accuracy: 0.4064 - loss: 1.6356
Epoch 2/10
782/782 - 2s - 3ms/step - accuracy: 0.5311 - loss: 1.3143
Epoch 3/10
782/782 - 2s - 3ms/step - accuracy: 0.5842 - loss: 1.1809
Epoch 4/10
782/782 - 2s - 3ms/step - accuracy: 0.6213 - loss: 1.0840
Epoch 5/10
782/782 - 2s - 3ms/step - accuracy: 0.6484 - loss: 1.0096
Epoch 6/10
782/782 - 2s - 3ms/step - accuracy: 0.6692 - loss: 0.9508
Epoch 7/10
782/782 - 2s - 3ms/step - accuracy: 0.6882 - loss: 0.8990
Epoch 8/10
782/782 - 2s - 3ms/step - accuracy: 0.7084 - loss: 0.8501
Epoch 9/10
782/782 - 2s - 3ms/step - accuracy: 0.7195 - loss: 0.8110
Epoch 10/10
782/782 - 2s - 3ms/step - accuracy: 0.7336 - loss: 0.7719


2025-05-24 12:53:44.483756: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,30,30]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.526, f32[32,3,3,3]{3,2,1,0} %bitcast.533, f32[32]{0} %bitcast.535), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="sequential_1/conv2d_1/convolution" source_file="/project/nanocourse/SWE_OOP/shared/CondaEnvs/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0},"force_earliest_schedule":false}
2025-05-24 12:53:44.493138: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted pote

157/157 - 1s - 9ms/step - accuracy: 0.7017 - loss: 0.8714


[0.8714316487312317, 0.70169997215271]

In [7]:
def my_model():
    inputs = keras.Input(shape=(32, 32, 3))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


modelFunc = my_model()
modelFunc.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [8]:
print(modelFunc.summary())

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 13, 13, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 4, 4, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       131,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 225,930 (882.54 KB)

 Trainable params: 225,482 (880.79 KB)

 Non-trainable params: 448 (1.75 KB)

None


In [9]:
modelFunc.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
modelFunc.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
782/782 - 9s - 11ms/step - accuracy: 0.5187 - loss: 1.3484
Epoch 2/10
782/782 - 3s - 3ms/step - accuracy: 0.6601 - loss: 0.9722
Epoch 3/10
782/782 - 3s - 3ms/step - accuracy: 0.7162 - loss: 0.8202
Epoch 4/10
782/782 - 3s - 3ms/step - accuracy: 0.7529 - loss: 0.7138
Epoch 5/10
782/782 - 3s - 3ms/step - accuracy: 0.7824 - loss: 0.6285
Epoch 6/10
782/782 - 3s - 3ms/step - accuracy: 0.8111 - loss: 0.5545
Epoch 7/10
782/782 - 3s - 3ms/step - accuracy: 0.8315 - loss: 0.4912
Epoch 8/10
782/782 - 3s - 3ms/step - accuracy: 0.8490 - loss: 0.4373
Epoch 9/10
782/782 - 3s - 3ms/step - accuracy: 0.8699 - loss: 0.3826
Epoch 10/10
782/782 - 3s - 3ms/step - accuracy: 0.8844 - loss: 0.3404
157/157 - 1s - 9ms/step - accuracy: 0.6975 - loss: 0.9679


[0.9679063558578491, 0.6974999904632568]

In [10]:
# To Do:
# 1. construct a new class (type) called MyModelSubClass which is a keras.Model, instantiate it, 
# and then fit the model to the data. 
#
# 2. Now explore: See what kind of accuracy you can get by training longer, 
# increasing model size, changing the kernel sizes.


In [17]:
class MyModelSubClass(keras.Model):
    def __init__(self,name='MyModelSubclass', **kwargs):
        # call super (base) class's init
        super().__init__(name=name, **kwargs)

        # define layers to be used in forward pass
        self.conv1 = layers.Conv2D(32, 3,padding='same') # Added padding for example
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()
        self.pool1 = layers.MaxPooling2D()

        self.conv2 = layers.Conv2D(64, 3,padding='same')
        self.bn2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU()
        self.pool2 = layers.MaxPooling2D()

        self.conv3 = layers.Conv2D(128, 3, padding='same')
        self.bn3 = layers.BatchNormalization()
        self.relu3 = layers.ReLU()
        # Using tf.nn.relu directly in call, so no relu layer needed here if preferred

        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(64, activation="relu") # This is your intermediate dense layer
        self.output_dense = layers.Dense(10)    # Final output layer
        
    def call(self, inputs, training=None):
        
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = keras.activations.relu(x) # Using the ReLU layer
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = keras.activations.relu(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)
        x = keras.activations.relu(x)

        x = self.flatten(x)
        x = self.dense1(x)
        outputs = self.output_dense(x) 
        # Return the prediction
        return outputs
    
    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [18]:
# Instantiate the model
test = MyModelSubClass(name = 'cookie')


In [19]:
# Print a summary of the model
my_model = test.model()
print(my_model.summary())

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_12 (ReLU)                 │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_13 (ReLU)                 │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_14 (ReLU)                 │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │       524,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 619,146 (2.36 MB)

 Trainable params: 618,698 (2.36 MB)

 Non-trainable params: 448 (1.75 KB)

None


In [20]:
# Compile the model
my_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)


In [21]:
# Fit the model
my_model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)


Epoch 1/10


/project/nanocourse/SWE_OOP/shared/CondaEnvs/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_39']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)
2025-05-24 13:03:26.996055: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.9 = (f32[64,32,32,32]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,3,32,32]{3,2,1,0} %bitcast.3599, f32[32,3,3,3]{3,2,1,0} %bitcast.3606, f32[32]{0} %bitcast.4510), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="functional_3_1/conv2d_9_1/convolution" source_file="/project/nanocourse/SWE_OOP/shared/CondaEnvs/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id

782/782 - 9s - 12ms/step - accuracy: 0.5590 - loss: 1.2412
Epoch 2/10
782/782 - 3s - 4ms/step - accuracy: 0.6938 - loss: 0.8719
Epoch 3/10
782/782 - 3s - 4ms/step - accuracy: 0.7476 - loss: 0.7230
Epoch 4/10
782/782 - 3s - 4ms/step - accuracy: 0.7839 - loss: 0.6223
Epoch 5/10
782/782 - 3s - 4ms/step - accuracy: 0.8130 - loss: 0.5360
Epoch 6/10
782/782 - 3s - 4ms/step - accuracy: 0.8389 - loss: 0.4680
Epoch 7/10
782/782 - 3s - 4ms/step - accuracy: 0.8607 - loss: 0.4048
Epoch 8/10
782/782 - 3s - 4ms/step - accuracy: 0.8823 - loss: 0.3462
Epoch 9/10
782/782 - 3s - 4ms/step - accuracy: 0.9022 - loss: 0.2915
Epoch 10/10
782/782 - 3s - 4ms/step - accuracy: 0.9164 - loss: 0.2488


In [22]:
# Check your model's performance
my_model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 2s - 10ms/step - accuracy: 0.7210 - loss: 0.9747


[0.9747118353843689, 0.7210000157356262]

In [ ]:
### Rinse and repeat; this time add more layers. 

In [23]:
class MyModelSubClass_Layers(keras.Model):
    def __init__(self,name='MyModelSubClass_Layers', **kwargs):
        # call super (base) class's init
        super().__init__(name=name, **kwargs)

        # define layers to be used in forward pass
        self.conv1 = layers.Conv2D(32, 3, padding='same') # Added padding for example
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()
        self.pool1 = layers.MaxPooling2D()

        self.conv2 = layers.Conv2D(64, 3)
        self.bn2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU()
        self.pool2 = layers.MaxPooling2D()

        self.conv3 = layers.Conv2D(128, 3, padding='same')
        self.bn3 = layers.BatchNormalization()
        self.relu3 = layers.ReLU()
        self.pool3 = layers.MaxPooling2D()
        
        self.conv4 = layers.Conv2D(256, 3, padding='same')
        self.bn4 = layers.BatchNormalization()
        self.relu4 = layers.ReLU()
        # Using tf.nn.relu directly in call, so no relu layer needed here if preferred

        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(64, activation="relu") # This is your intermediate dense layer
        self.output_dense = layers.Dense(10)    # Final output layer
        
    def call(self, inputs, training=None):
        
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x) # Using the ReLU layer
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = self.bn4(x, training=training)
        x = self.relu4(x)

        x = self.flatten(x)
        x = self.dense1(x)
        outputs = self.output_dense(x) 
        # Return the prediction
        return outputs
    
    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [24]:
# Instantiate the model
ctl = MyModelSubClass_Layers()

In [25]:
# Print a summary of the model
tl = ctl.model()
print(tl.summary())

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_15 (ReLU)                 │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_16 (ReLU)                 │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_17 (ReLU)                 │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 3, 3, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_18 (ReLU)                 │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 538,506 (2.05 MB)

 Trainable params: 537,546 (2.05 MB)

 Non-trainable params: 960 (3.75 KB)

None


In [26]:
# Compile the model
tl.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [27]:
# Fit the model
tl.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10


/project/nanocourse/SWE_OOP/shared/CondaEnvs/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_54']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)
2025-05-24 13:05:36.263650: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:549] Omitted potentially buggy algorithm eng14{} for conv %cudnn-conv-bias-activation.13 = (f32[64,64,14,14]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,32,16,16]{3,2,1,0} %bitcast.5822, f32[64,32,3,3]{3,2,1,0} %bitcast.4589, f32[64]{0} %bitcast.5882), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", metadata={op_type="Conv2D" op_name="functional_4_1/conv2d_13_1/convolution" source_file="/project/nanocourse/SWE_OOP/shared/CondaEnvs/lib/python3.12/site-packages/tensorflow/python/framework/ops.py" source_line=1200}, backend_config={"operation_queue_id":"0","w

782/782 - 11s - 15ms/step - accuracy: 0.5550 - loss: 1.2503
Epoch 2/10
782/782 - 3s - 4ms/step - accuracy: 0.7022 - loss: 0.8536
Epoch 3/10
782/782 - 3s - 4ms/step - accuracy: 0.7618 - loss: 0.6793
Epoch 4/10
782/782 - 3s - 4ms/step - accuracy: 0.8049 - loss: 0.5631
Epoch 5/10
782/782 - 3s - 4ms/step - accuracy: 0.8397 - loss: 0.4623
Epoch 6/10
782/782 - 3s - 4ms/step - accuracy: 0.8699 - loss: 0.3789
Epoch 7/10
782/782 - 3s - 4ms/step - accuracy: 0.8961 - loss: 0.3046
Epoch 8/10
782/782 - 3s - 4ms/step - accuracy: 0.9213 - loss: 0.2338
Epoch 9/10
782/782 - 3s - 4ms/step - accuracy: 0.9381 - loss: 0.1862
Epoch 10/10
782/782 - 3s - 4ms/step - accuracy: 0.9549 - loss: 0.1399


In [28]:
# Check your model's performance. How does the performance compare with the smaller model?
tl.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 2s - 11ms/step - accuracy: 0.7415 - loss: 0.9790


[0.979032039642334, 0.7415000200271606]

In [ ]:
### Rinse and repeat; this time change the kernel size. 

In [29]:
class MyModelSubClass_Kernel(keras.Model):
    def __init__(self,name='MyModelSubClass_Kernel', **kwargs):
        # call super (base) class's init
        super().__init__(name=name, **kwargs)

        # define layers to be used in forward pass
        self.conv1 = layers.Conv2D(32, 5, padding='same') # Added padding for example
        self.bn1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()
        self.pool1 = layers.MaxPooling2D()

        self.conv2 = layers.Conv2D(64, 5)
        self.bn2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU()
        self.pool2 = layers.MaxPooling2D()

        self.conv3 = layers.Conv2D(128, 5, padding='same')
        self.bn3 = layers.BatchNormalization()
        self.relu3 = layers.ReLU()
        self.pool3 = layers.MaxPooling2D()
        
        self.conv4 = layers.Conv2D(256, 5, padding='same')
        self.bn4 = layers.BatchNormalization()
        self.relu4 = layers.ReLU()
        # Using tf.nn.relu directly in call, so no relu layer needed here if preferred

        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(64, activation="relu") # This is your intermediate dense layer
        self.output_dense = layers.Dense(10)    # Final output layer
        
    def call(self, inputs, training=None):
        
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu1(x) # Using the ReLU layer
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.bn3(x, training=training)
        x = self.relu3(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = self.bn4(x, training=training)
        x = self.relu4(x)

        x = self.flatten(x)
        x = self.dense1(x)
        outputs = self.output_dense(x) 
        # Return the prediction
        return outputs
    
    def model(self):
        x = keras.Input(shape=(32, 32, 3))
        return keras.Model(inputs=[x], outputs=self.call(x))


In [30]:
# Instantiate the model
ctk = MyModelSubClass_Layers()

In [31]:
# Print a summary of the model
tk = ctk.model()
print(tk.summary())

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_19 (ReLU)                 │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_20 (ReLU)                 │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_21 (ReLU)                 │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 3, 3, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_22 (ReLU)                 │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 538,506 (2.05 MB)

 Trainable params: 537,546 (2.05 MB)

 Non-trainable params: 960 (3.75 KB)

None


In [32]:
# Compile the model
tk.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    metrics=["accuracy"],
)

In [33]:
# Fit the model
tk.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)

Epoch 1/10


/project/nanocourse/SWE_OOP/shared/CondaEnvs/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_73']
Received: inputs=Tensor(shape=(None, 32, 32, 3))
  warnings.warn(msg)
2025-05-24 13:12:43.963038: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'loop_add_fusion', 4 bytes spill stores, 4 bytes spill loads



782/782 - 11s - 14ms/step - accuracy: 0.5480 - loss: 1.2649
Epoch 2/10
782/782 - 3s - 4ms/step - accuracy: 0.6934 - loss: 0.8710
Epoch 3/10
782/782 - 3s - 4ms/step - accuracy: 0.7555 - loss: 0.7003
Epoch 4/10
782/782 - 3s - 4ms/step - accuracy: 0.7979 - loss: 0.5758
Epoch 5/10
782/782 - 3s - 4ms/step - accuracy: 0.8350 - loss: 0.4718
Epoch 6/10
782/782 - 3s - 4ms/step - accuracy: 0.8676 - loss: 0.3843
Epoch 7/10
782/782 - 3s - 4ms/step - accuracy: 0.8908 - loss: 0.3143
Epoch 8/10
782/782 - 3s - 4ms/step - accuracy: 0.9155 - loss: 0.2462
Epoch 9/10
782/782 - 3s - 4ms/step - accuracy: 0.9337 - loss: 0.1960
Epoch 10/10
782/782 - 3s - 4ms/step - accuracy: 0.9470 - loss: 0.1603


In [34]:
# Check your model's performance. How does the performance compare with the other two models?
tk.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 2s - 13ms/step - accuracy: 0.7393 - loss: 1.0349


[1.0348734855651855, 0.739300012588501]